In [1]:
# coding: utf-8
'''
-----------
To Do :
  - parameter shoul be read from a file isntead of dictionary!
  - finding a general way to define Q and R according to number of pendulums
  - (should I have a dynamic way of changing  Q and R to find the best result ?! )
'''
#=============================================================
# Standard Python modules
#=============================================================
import os, sys
import dill
import logging

#=============================================================
# External Python modules
#=============================================================
#from __future__ import division, print_function
from sympy.physics.vector import init_vprinting, vlatex
init_vprinting(use_latex='mathjax', pretty_print=False)

import sympy as sm
import sympy.physics.mechanics as me
import numpy as np
from numpy.linalg import inv as np_inv
import scipy as sc
from scipy.integrate import odeint

from odeintw import odeintw

#import ipydex
#=============================================================
# Standard Python modules
#=============================================================
from functions import *
from sys_model import *
from traj_opt import *
#=============================================================
# Lqr control for top equilibrium point
#=============================================================

logging.basicConfig(
    filename='pen_odeint.log',
    level=logging.DEBUG,
    format='%(asctime)s %(name)-12s %(levelname)-8s %(message)s')

# logger = logging.getLogger()
# handler = logging.StreamHandler()
# formatter = logging.Formatter(
#     '%(asctime)s %(name)-12s %(levelname)-8s %(message)s')
# handler.setFormatter(formatter)
# logger.addHandler(handler)
# logger.setLevel(logging.DEBUG)

#ipydex.activate_ips_on_exception()

# Defining equilibrium point and system parameters
if n == 1:
    param_values = config.parameter_values_simple_pendulum
    x0 = [0., 0., 0., 0.]

elif n == 2:
    param_values = config.parameter_values_double_pendulum
    x0 = [0., 0., 0., 0., 0., 0.]

elif n == 3:
    param_values = config.parameter_values_triple_pendulum
    x0 = [0., 0., 0., 0., 0., 0., 0., 0.]

param_symb = list(a + d + m + J + (g, f))
param_list = zip(param_symb, param_values)

u0 = [0.0]
equilibrium_point = x0 + u0

# linearization of model @ equilibrium point
A, B = linearize_state_equ(fx, gx, q, qdot, u, param_list, equilibrium_point)

# lqr control to find K (we need it as a boundry-value (Pe) for our tracking !)
Q = np.identity(4)
R = 0.01 * np.identity(1)
# k_top = lqr(A, B, Q, R)
results = lqr(A, B, Q, R, additional_outputs=True)
k_top = results[0]
'''
# k(t)= R^-1 * B.T * P(t)  ---> Pe=(B.T).inv() * R * k_top
# ATTENTION :
#             - B is a 4*1 matrix so we couldnt find Pe exactly using
#               equation above! We impose a second condition for Pe
#               to be a diagonal Matrix. --> Pe=diag(p1, p2, p3, p4)
#
#             - On the other hand because the first and second
#               component of the vector B are always 0 , finding p1, p2
#               provide no additional informations. so we could set
#               them to be 0 and just find p3 and p4 !

diag_Pe= [0.0 for i in range(len(q))]
for i in range(len(q)):
      indx=i+len(q)
      diag_Pe.append(k_top[0,indx]*R[0,0]/B[indx,0])

Pe=np.diag(diag_Pe)
'''

Pe = results[1].reshape(16)

# lqr tracking of the desired trajectory :
# determining linearized model at any equilibrium point
A_eq, B_eq = linearize_state_equ(fx, gx, q, qdot, u, param_list)

# solving riccati differential equations inverse in time :
dynamic_symbs = q + qdot + [u]

frames_per_sec = 15
final_time = 2
t = np.linspace(0.0, final_time, final_time * frames_per_sec)

# P_dot= riccati_diff_equ(Pe, final_time, A_eq, B_eq, Q, R, dynamic_symbs)
Pe0 = np.identity(4)
# converting Matrix to Vector to be able to use in odeint
#   we're converting it back again to a Matrix ! (it also need some changes
#   in riccati_diff_equ function !)
Pe0 = Pe0.reshape(16)
# logger.debug('Pe = y0 : %f', Pe0)
# print ('Pe = y0 :', Pe0)
'''
P = odeint(riccati_diff_equ, Pe, t[::-1], args=(A_eq, B_eq, Q, R, dynamic_symbs))

with open('P_matrix.pkl','wb' ) as file :
    dill.dump(P, file)
'''
with open('P_matrix.pkl', 'rb') as file :
    P=dill.load(file)

# finding gain k for lqr Tracking :

Psim=P[::-1]
K_matrix= generate_gain_matrix(R, B_eq, Psim, t, dynamic_symbs)

# finding states of the system using calculated K_matrix and
# comparing the results with desired trajecory !
xdot_func= sympy_states_to_func(dynamic_symbs, param_list)
#ipydex.IPS()


def ode_function(x, t, xdot_func, K_matrix, Vect,  mode='Open_loop'):
    '''
    it's the dy/dt=func(y, t, args)  to be used in odeint
    (the first two arguments are system state x and time t)

    there are two modes available:
     - Open_loop is defualt and use for tracking (Folgerung)
     - Closed_loop could be activated in mode and could be 
       used as regulator (Regler)

    ATTENTION :
      - use sympy_states_to_func to produce xdot functions out of 
        sympy expresisons. 
        (you have to run sympy_state_to_func once and store the result
        so you could pass it as xdot_func )

    '''
    
    logging.debug('x_new: %s \n \n', x)
    logging.debug('----------------------------------------------------------------')
    n=len(Vect)
    xs=config.cs_ret[0](t)
    us=config.cs_ret[1](t)
    
    if mode == 'Open_loop' :
        k0 = np.interp(t, Vect, K_matrix[:, 0])
        k1 = np.interp(t, Vect, K_matrix[:, 1])
        k2 = np.interp(t, Vect, K_matrix[:, 2])
        k3 = np.interp(t, Vect, K_matrix[:, 3])

        k = np.array([k0, k1, k2, k3])
        delta_x = x - xs
        delta_u = (-1) * k.T.dot(delta_x)
        inputs = us + delta_u
        # loggings :
        logging.debug('k :%s', k)
        logging.debug('delta_x: %s',delta_x)
        logging.debug('delta_u: %s \n', delta_u)
    elif mode == 'Closed_loop' :
        inputs= us

    state= x
    logging.debug('t: %s \n', t)
    
    logging.debug('us: %s', us )
    logging.debug('xs:%s \n',  xs)
    logging.debug('state: %s', state)
    logging.debug('inputs: %s \n', inputs)
    
    # ipydex.IPS()
    xdot= xdot_func(state, inputs)
    logging.debug('x_current: %s', x)    
    logging.debug('xdot : %s ', xdot)
    
    return xdot


# x_open_loop= odeint(ode_function, xa, t, args=(xdot_func, K_matrix, t) )
# x_closed_loop= odeint(ode_function, xa, t, args=(xdot_func, K_matrix, t, 'Closed_loop') )
#ipydex.IPS()

'''
x_closed_loop=np.array(config.cs_ret[0](t).tolist())

import matplotlib as mpl
import matplotlib.pyplot as plt

fig, axes = plt.subplots()
axes.plot(t, x_open_loop[:, 1] * 180 / np.pi, 'o')
axes.plot(t, x_closed_loop[:]
axes.plot(vect, K)

plt.show()

'''

2018-06-01 15:22:17,859 matplotlib   DEBUG    $HOME=C:\Users\kuros
2018-06-01 15:22:17,864 matplotlib   DEBUG    matplotlib data path C:\Users\kuros\Anaconda3\envs\py2env\lib\site-packages\matplotlib\mpl-data
2018-06-01 15:22:17,898 matplotlib   DEBUG    loaded rc file C:\Users\kuros\Anaconda3\envs\py2env\lib\site-packages\matplotlib\mpl-data\matplotlibrc
2018-06-01 15:22:17,908 matplotlib   DEBUG    matplotlib version 2.2.2
2018-06-01 15:22:17,910 matplotlib   DEBUG    interactive is False
2018-06-01 15:22:17,911 matplotlib   DEBUG    platform is win32
2018-06-01 15:22:17,913 matplotlib   DEBUG    loaded modules: ['sympy.functions.combinatorial.factorials', 'numpy.core.info', 'traitlets.config.decorator', 'sympy.sets.__future__', 'distutils.sysconfig', 'pygments.filters', 'IPython.utils.pprint', 'logging.weakref', 'mpmath.libmp.StringIO', 'dateutil.tz.os', 'pprint', 'prompt_toolkit.key_binding.bindings.vi', 'ipykernel.traitlets', 'dill.objtypes', 'IPython.terminal.sys', 'sympy.interac

2018-06-01 15:22:18,043 matplotlib   DEBUG    CACHEDIR=C:\Users\kuros\.matplotlib
2018-06-01 15:22:18,075 matplotlib.font_manager DEBUG    Using fontManager instance from C:\Users\kuros\.matplotlib\fontList.json
2018-06-01 15:22:18,322 matplotlib.backends DEBUG    backend module://ipykernel.pylab.backend_inline version unknown
2018-06-01 15:22:18,615 root         WARNING  scipy version (1.1.0) may be out of date


2018-06-01 15:22:18,617 root         WARNING  sympy version (1.1.1) may be out of date


2018-06-01 15:22:18,694 root         DEBUG    Determine system/input dimensions
2018-06-01 15:22:18,711 root         DEBUG    --> state: 4
2018-06-01 15:22:18,713 root         DEBUG    --> input: 1
2018-06-01 15:22:18,713 root         DEBUG    --> a.f.p.: 0
2018-06-01 15:22:18,714 root         DEBUG    --> penalties: 0
2018-06-01 15:22:18,907 root         DEBUG    Looking for integrator chains
2018-06-01 15:22:18,915 root         DEBUG    --> found: x1 -> x3 -> u1
2018-06-01 15:22:18,917 root         DEBUG    --> found: x2 -> x4
2018-06-01 15:22:18,918 root         INFO     Iteration #1; spline parts_ 10


INFO: 	 Iteration #1; spline parts_ 10


...

2018-06-01 15:22:30,390 root         INFO     par = []


INFO: 	 par = []
Pytrajecotry Succeeded!


2018-06-01 15:22:31,476 root         DEBUG    u : u
2018-06-01 15:22:31,483 root         DEBUG    fx_expr: Matrix([[qdot0(t)], [qdot1(t)], [0], [a0*g*m1*sin(q1(t))/(J1 + a0**2*m1) + d1*qdot1(t)/(J1 + a0**2*m1)]])
2018-06-01 15:22:31,490 root         DEBUG    gx_expr: Matrix([[0.0], [0.0], [1], [-a0*m1*cos(q1(t))/(J1 + a0**2*m1)]])
2018-06-01 15:22:31,522 root         DEBUG    xdot_exr: Matrix([[qdot0(t)], [qdot1(t)], [u], [-3.16149160600208*u*cos(q1(t)) + 0.185707918585648*qdot1(t) + 31.0142326548804*sin(q1(t))]])


"\nx_closed_loop=np.array(config.cs_ret[0](t).tolist())\n\nimport matplotlib as mpl\nimport matplotlib.pyplot as plt\n\nfig, axes = plt.subplots()\naxes.plot(t, x_open_loop[:, 1] * 180 / np.pi, 'o')\naxes.plot(t, x_closed_loop[:]\naxes.plot(vect, K)\n\nplt.show()\n\n"

2018-06-01 15:22:31,601 matplotlib.backends DEBUG    backend module://ipykernel.pylab.backend_inline version unknown


In [22]:
#=============================================================
# Standard Python modules
#=============================================================
import os, sys
import dill
import logging

#=============================================================
# External Python modules
#=============================================================
#from __future__ import division, print_function
from sympy.physics.vector import init_vprinting, vlatex
init_vprinting(use_latex='mathjax', pretty_print=False)

import sympy as sm
import sympy.physics.mechanics as me
import numpy as np
from numpy.linalg import inv as np_inv
import scipy as sc
from scipy.integrate import odeint

from odeintw import odeintw

#import ipydex
#=============================================================
# Standard Python modules
#=============================================================
from functions import *
from sys_model import *

In [26]:
qdd_expr = fx + gx * u

In [34]:
qdd_func = [
            sm.lambdify(q+qdot+[u],
                        qdd_expr[i + len(q)], 'sympy') for i in range(len(q))
        ]

In [35]:
qdd_func[1](0,1,2,3,4)

a0*g*m1*sin(1)/(J1 + a0**2*m1) - 4*a0*m1*cos(1)/(J1 + a0**2*m1) + 3*d1/(J1 + a0**2*m1)

In [33]:
len(q)

2

In [36]:
 qdd_func = [
        sm.lambdify(q + qdot + [u], qdd_expr[i + len(q)], 'sympy')
        for i in range(len(q))
    ]

In [38]:
qdd_func[0](0,1,2,3,4)

4

In [39]:
q

[q0, q1]

In [45]:
x0=[0 for i in range(len(b))]


In [46]:
x0

[0, 0, 0, 0]

In [48]:
x0[1:3]=5

TypeError: can only assign an iterable

In [43]:
x0

[0, 5]

In [44]:
b

[b0, b1, b2, b3]

In [53]:
x0=[0]+ [np.pi in range(len(b)-1)]+ [0  in range(len(q))]

In [54]:
x0

[0, False, True]